# Collect results

In [ ]:
import load

results = load.Results(su)
#%%time
# for n in [5, 10, 20]:
#   sols = results.collect_and_save(n, linkage_map.keys())
_
# for n in [5, 10, 20]:
#   sols = results.collect_and_save(n, ["rf"])

In [3]:
import os
import pandas as pd
import scoring
from io_utils import sort_genes
from tqdm import tqdm

RUNS_DIR = os.path.join("..", "runs")
EVAL_DIR = os.path.join("..", "eval")
data_dir = "../data/"

In [2]:
df = pd.read_csv(data_dir + "data_unscaled.csv")
df
display(df.head())

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from scoring import Scoring

S = Scoring()

In [16]:
def _clean_and_eval(dfs, num_genes, runs):
        for df, run in zip(dfs, runs):
            df['run_id'] = run.split('/')[-1]
        conc = pd.concat(dfs, ignore_index=True)
        sols = sort_genes(conc[[str(i) for i in range(num_genes)]])
        metrics = conc[conc.columns.intersection(scoring.abs_linkages + scoring.log_linkages)]#.reset_index()
        sols = sols.join(metrics)
        
        x = len(sols)
        sols = sols.drop_duplicates(subset=[str(i) for i in range(num_genes)])
        #metrics = sols.apply(lambda x: S.score(x, log=True), axis=1)
        # metrics_log = sols.apply(lambda x: self.scoring.score(x, log=True), axis=1)
        #s#ols = sols.join(metrics)
        #sols = sols.join(conc['run_id'])
        sols = sols.sort_values(by="single_log", ascending=False)
        return sols

In [17]:
def collect_and_save(num_genes, metrics):
        runs = next(os.walk(RUNS_DIR))[1]
        runs = list(
            filter(
                lambda x: f"_{num_genes}" in x and any(metric in x for metric in metrics),
                runs,
            )
        )
        print(runs)

        runs = [os.path.join(RUNS_DIR, run_dir, "best_sols.csv") for run_dir in runs]
        dfs = []
        for run in tqdm(runs):
            try:
                dfs.append(pd.read_csv(run))
            except FileNotFoundError:
                print('Run is empty: ', run)

        sols = _clean_and_eval(dfs, num_genes, runs)
        sols.to_csv(os.path.join(EVAL_DIR, f'{num_genes}_{"_".join(metrics)}.csv'))
        return sols

In [19]:
%%time

for n in tqdm(range(4,21)):
    print('-------- ', n, ' -------')
    collect_and_save(n, ['single'])
    
#collect_and_save(20, ['single'])

  0%|                                                 | 0/17 [00:00<?, ?it/s]

--------  4  -------
['27_single_4', '28_single_4', '30_single_4', '31_single_4', '32_single_4', '33_single_4', '34_single_4']



  6%|██▍                                      | 1/17 [00:00<00:02,  7.33it/s]

Run is empty:  ..\runs\27_single_4\best_sols.csv
Run is empty:  ..\runs\28_single_4\best_sols.csv
--------  5  -------
['10_single_5', '1_single_5', '22_single_5', '23_single_5', '24_single_5', '25_single_5', '26_single_5', '2_single_5', '6_single_5', '7_single_5', '8_single_5', '9_single_5']



 12%|████▊                                    | 2/17 [00:00<00:07,  1.98it/s]

...

--------  19  -------
['100_single_19', '101_single_19', '97_single_19', '98_single_19', '99_single_19']



 94%|█████████████████████████████████████▋  | 16/17 [00:03<00:00,  3.43it/s]

--------  20  -------
['102_single_20', '103_single_20', '104_single_20', '105_single_20', '17_single_20', '18_single_20', '19_single_20', '20_single_20', '21_single_20']



100%|████████████████████████████████████████| 17/17 [00:04<00:00,  3.97it/s]

Wall time: 4.29 s


In [ ]:
%%time

from scoring import linkage_map_log
for n in [5]:
    sols = results.collect_and_save(n, linkage_map_log.keys())

In [ ]:
sols

In [ ]:
%%time

from scoring import linkage_map_log
for n in [10]:
    sols = results.collect_and_save(n, linkage_map_log.keys())
sols